# Imports and loading data

In [1]:
import pickle
import pandas as pd
import vrme_lib

## Loading data from VRM-E notebook

In [2]:
with open('significance_data_vrme.pkl', 'rb') as f:
  vrme_objs = pickle.load(f)

vrme_df_embeddings_2017 = vrme_objs['df_embeddings_2017']
vrme_df_embeddings_2018 = vrme_objs['df_embeddings_2018']
vrme_df_embeddings_2017['match_ave_rating'] = vrme_df_embeddings_2017.apply(lambda row: vrme_lib.lambda_get_match_potential_outcomes(row, vrme_df_embeddings_2018), axis =1)
vrme_df_embeddings_2017['diff_2018_2017'] = vrme_df_embeddings_2017['match_ave_rating'] - vrme_df_embeddings_2017['AVG_rating']

## Loading data from SPSM notebook

In [3]:
with open('significance_data_spsm.pkl', 'rb') as f:
  vrme_objs = pickle.load(f)

df_paper_strata = vrme_objs['df_paper_strata']

# VRM-E v/s Naive


Step 1: Generate bootstrap samples from 2017 data

Step 2: Get ATC from bootstrap samples for VRM-E

Step 3: Get ATC for naive bootstrap: Average(Boot_strap 2017 samples) - Average(2018 Empirical Samples

Step 4: Follow figure 1 from https://aclanthology.org/D12-1091.pdf

Delta(Bootstrap) = ATC_VRM-E(Bootstrap) - ATC_naive(Boostrap)

Delta(X) = ATC_VRM-E  - ATC_Naive = -0.17 - -0.25 = 0.08

In [4]:
average_rating_2018 = vrme_df_embeddings_2018.AVG_rating.mean()
delta_x = 0.08
s = 0
B = 5000
for n in range(B):
    bootstrap_sample = vrme_df_embeddings_2017.sample(n=vrme_df_embeddings_2017.shape[0], replace = True, random_state=n)
    ATC_VRM_E = bootstrap_sample.diff_2018_2017.mean()

    ATC_Naive = average_rating_2018 - bootstrap_sample.AVG_rating.mean()

    delta_bootstrap = ATC_VRM_E - ATC_Naive

    if delta_bootstrap > 2*delta_x:
        s += 1

print('P-Value for VRM-E vs Naive is: ',s/B)

P-Value for VRM-E vs Naive is:  0.0


# VRM-E v/s SPSM

Step 1: Generate bootstrap samples from 2017 data

Step 2: Get ATC from bootstrap samples for VRM-E

Step 3: Get ATC for naive bootstrap: Average(Boot_strap 2017 samples) - Average(2018 Empirical Samples

Step 4: Follow figure 1 from https://aclanthology.org/D12-1091.pdf

Delta(Bootstrap) = ATC_VRM-E(Bootstrap) - ATC_SPSM(Boostrap)

Delta(X) = ATC_VRM-E  - ATC_SPSM = -0.17 - -0.26 = 0.09

In [5]:
#data processing to get the propensity and strata
vrme_df_embeddings_2017 = pd.merge(vrme_df_embeddings_2017, df_paper_strata[df_paper_strata['treatment']==1][['paper_id','pscore','strata','treatment']], on='paper_id')

delta_x2 = 0.09
s2 = 0
B2 = 5000

for n in range(B2):
    bootstrap_sample = vrme_df_embeddings_2017.sample(n=vrme_df_embeddings_2017.shape[0], replace = True, random_state=n)
    ATC_VRM_E = bootstrap_sample.diff_2018_2017.mean()

    strata_2_weight = bootstrap_sample[(bootstrap_sample['strata'] ==2) & (bootstrap_sample['treatment'] ==1)].shape[0]/490
    strata_3_weight = bootstrap_sample[(bootstrap_sample['strata'] ==3) & (bootstrap_sample['treatment'] ==1)].shape[0]/490
    strata_4_weight = bootstrap_sample[(bootstrap_sample['strata'] ==4) & (bootstrap_sample['treatment'] ==1)].shape[0]/490

    strat_2_treated = bootstrap_sample[(bootstrap_sample['strata'] ==2) & (bootstrap_sample['treatment'] ==1)]['AVG_rating'].mean()
    strat_2_control = df_paper_strata[(df_paper_strata['strata'] ==2) & (df_paper_strata['treatment'] ==0)]['AVG_rating'].mean()


    strat_3_treated = bootstrap_sample[(bootstrap_sample['strata'] ==3) & (bootstrap_sample['treatment'] ==1)]['AVG_rating'].mean()
    strat_3_control = df_paper_strata[(df_paper_strata['strata'] ==3) & (df_paper_strata['treatment'] ==0)]['AVG_rating'].mean()

    strat_4_treated = bootstrap_sample[(bootstrap_sample['strata'] ==4) & (bootstrap_sample['treatment'] ==1)]['AVG_rating'].mean()
    strat_4_control = df_paper_strata[(df_paper_strata['strata'] ==4) & (df_paper_strata['treatment'] ==0)]['AVG_rating'].mean()



    ATC_SPSM = (strat_4_treated - strat_4_control)* strata_4_weight + (strat_3_treated - strat_3_control)* strata_3_weight + (strat_2_treated - strat_2_control)* strata_2_weight
    #treatment and control were flipped for SPSM procedure
    ATC_SPSM = -ATC_SPSM

    delta_bootstrap = ATC_VRM_E - ATC_SPSM

    if delta_bootstrap > 2 * delta_x2:
        s2 += 1
    
print('P-Value for VRM-E vs SPSM is: ',s2/B2)

P-Value for VRM-E vs SPSM is:  0.0
